# Fund Strategy - Signal Event 0001

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2015-12-31'
ed = '2026-02-13'

In [3]:
instrument_list = ['510170.SH', '513520.SH', '513030.SH']

In [4]:
sql = f"""
SELECT
    date,
    instrument,
    m_avg(close, 10) AS ma_short,
    m_avg(close, 20) AS ma_long,
    IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_buy,
    IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_sell,
FROM cn_fund_bar1d
WHERE instrument IN {str(tuple(instrument_list))}
QUALIFY (signal_buy = 1 OR signal_sell = 1)
"""

In [5]:
df = dai.query(sql, filters = {"date":[sd, ed]}).df()
df

,date,instrument,ma_short,ma_long,signal_buy,signal_sell
0,2016-03-08,513030.SH,0.826300,0.825500,1,0
1,2016-04-08,513030.SH,0.856500,0.856650,0,1
2,2016-04-20,513030.SH,0.862300,0.860700,1,0
3,2016-05-13,513030.SH,0.882400,0.884750,0,1
4,2016-06-01,513030.SH,0.879400,0.877250,1,0
...,...,...,...,...,...,...
321,2025-09-24,510170.SH,4.430741,4.437012,0,1
322,2025-10-09,510170.SH,4.485357,4.467556,1,0
323,2025-11-24,510170.SH,4.874543,4.881016,0,1
324,2025-12-11,510170.SH,4.801723,4.791002,1,0


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3
    context.is_trade_by_weight = True
    context.weight = 1 / len(context.data['instrument'].unique())

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return

    instruments_buy  = set(df_now[df_now["signal_buy"]  == 1]["instrument"])
    instruments_sell = set(df_now[df_now["signal_sell"] == 1]["instrument"])
    instruments_hold = set(context.get_account_positions().keys())

    for instrument in instruments_buy - instruments_hold:
        if context.is_trade_by_weight:
            context.order_target_percent(instrument, context.weight)
        else:
            context.order(instrument, 100)
    
    for instrument in instruments_sell:
        context.order_target_percent(instrument, 0)

    for instrument in instruments_hold:
        if (data.current_dt - context.get_position(instrument).last_sale_date).days >= context.holding_days:
            context.order_target_percent(instrument, 0)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-19 15:06:36] [info     ] bigtrader.v34 开始运行 ..
[2026-02-19 15:06:37] [info     ] 2016-03-08, 2026-02-12, , , instruments=3
[2026-02-19 15:06:37] [info     ] bigtrader module V2.2.0
[2026-02-19 15:06:37] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-19 15:06:39] [info     ] backtest done, raw_perf_ds:dai.DataSource("_2f947c505e6d4a82b7ed25938cba3e6a")


[2026-02-19 15:06:40] [info     ] bigtrader.v34 运行完成 [3.849s].
